In [ ]:
pip install geopy
pip install lxml
pip install beautifulsoup4
from geopy.geocoders import Nominatim
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np

from pandas.io.json import json_normalize  # tranform JSON file into a pandas dataframe

import folium # map rendering library

# import k-means from clustering stage
from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
# Dataframe with 3 columns
# df = pd.DataFrame(res, columns = ["PostalCode", "Borough", "Neighborhood"])
# df.head()

df.head()

#Ignore cells with a borough that is Not assigned.

dfNotAssigned = df[df.Borough != 'Not assigned'].reset_index(drop=True)

dfNotAssigned.head()

dfcombined = dfNotAssigned.groupby(['Postal Code','Borough'], as_index=False).agg(lambda x: ','.join(x))
dfcombined.head()

NotNeighbourhood = dfcombined['Neighbourhood'] == "Not assigned"
dfcombined.loc[NotNeighbourhood, 'Neighbourhood'] = dfcombined.loc[NotNeighbourhood, 'Borough']

print("Shape after clean: ", dfcombined.shape)

print("Shape before clean: ", df.shape)

df_geo_coor = pd.read_csv("./Geospatial_Coordinates.csv")
df_geo_coor.head()

dfcombined2 = pd.merge(dfcombined, df_geo_coor, how='left', left_on = 'Postal Code', right_on = 'Postal Code')
# remove the "Postal Code" column
dfcombined2.drop("Postal Code", axis=1, inplace=True)
#dfcombined2 = df[df.Borough != 'Not assigned'].reset_index(drop=True)
#dfcombined2 = df[df.Neighbourhood != 'Not assigned'].reset_index(drop=True)
dfcombined2.head()

for lat, lng, borough, neighbourhood in zip(
        dfcombined2['Latitude'], 
        dfcombined2['Longitude'], 
        dfcombined2['Borough'], 
        dfcombined2['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

map_toronto